In [27]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re

#Data handling
from sklearn.model_selection import train_test_split

# Models 
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Normalizer
from sklearn.preprocessing import MinMaxScaler

#Pickling
from six.moves import cPickle as pickle

#Metrics
from sklearn.metrics import average_precision_score, recall_score, precision_score, f1_score

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline

x_train = []
y_train = []
x_test = []
y_test = []

In [28]:
def get_gender(text):
    text = text[len(text)-4]
    if('F' in text):
        return 'F'
    elif('M' in text):
        return 'M'
    else:
        return 'N/A'
    
def get_word_segments(align_file):
    segments = []
    with open(align_file,'r') as openFile:
        read_lines = openFile.readlines()
        lines = read_lines[1:len(read_lines)-1]
        for line in lines:
            splitted = line.split()
            segments.append({'sf': splitted[0], 'ef':splitted[1],'seg_scr':splitted[2],'word':splitted[3]})
    return segments
    
def generate_IEMOCAP_df():
    root_path = 'data/IEMOCAP_full_release/'
    folders = os.listdir(root_path)
    session_folders = filter(lambda x: x.startswith('Session'),folders)
    sentences_wav = '/sentences/wav/'
    forced_alignment = '/sentences/ForcedAlignment/'
    dialog_transcriptions = '/dialog/transcriptions/' #Ses01F_impro01.txt
    dialog_emo = '/dialog/EmoEvaluation/' #Ses01F_impro01.txt
    data = {}
    emotions = {}
    alignment = {}
    for session in session_folders:
        temp_root = root_path + session
        temp_conversations_path = temp_root + sentences_wav
        temp_transcripts_path = temp_root + dialog_transcriptions
        temp_alignments_path = temp_root + forced_alignment
        temp_emo_path = temp_root + dialog_emo
        emo_eval = os.listdir(temp_emo_path)
        for emo_file in emo_eval:
            emo_file_path = temp_emo_path + emo_file
            if(emo_file.endswith('.txt')):
                with open(emo_file_path,'r') as openFile:
                    for line in openFile.readlines():
                        if(line.startswith('[')):
                            line = line.split()
                            key = line[3]
                            emotion = line[4]
                            valence = re.search('[0-9]+.[0-9]+', line[5]).group(0)
                            arousal = re.search('[0-9]+.[0-9]+',line[6]).group(0)
                            dominance = re.search('[0-9]+.[0-9]+',line[7]).group(0)
                            emotions[key] = [emotion,valence,arousal, dominance]
        transcriptions = os.listdir(temp_transcripts_path)
        for transcript in transcriptions:
            transcript_path = temp_transcripts_path + transcript
            with open(transcript_path,'r') as openFile:
                for line in openFile.readlines():
                    if(line.startswith('Ses')):
                        line = line.split(' [')
                        key = line[0]
                        if('XX' not in key):
                            line = ' ['.join(line[1:])
                            line = line.split(']: ')
                            time = line[0]
                            script = ']: '.join(line[1:])
                            start_t, end_t = time.split('-')
                            data[key] = [start_t,end_t,script.split('\r\n')[0]]
        conversations = os.listdir(temp_conversations_path)
        for conversation in conversations:
            files_path = temp_conversations_path+ conversation
            alignments_path = temp_alignments_path + conversation
            files = os.listdir(files_path)
            for file in files:
                key,suf = file.split('.')
                if(suf == 'wav'):
                    data[key].append(files_path + '/'+ file)
                    data[key].append(alignments_path + '/'+key+'.wdseg')
    sentence_df = pd.DataFrame(data,index=['start_time','end_time','text','wav_path','alignment_path']).T
    emotion_df = pd.DataFrame(emotions, index=['emotion','valence','arousal','dominance']).T
    full = sentence_df.join(emotion_df)
    full = full.reset_index()
    full['gender'] = full['index'].apply(get_gender)
    return full

In [154]:
def log_reg(x_train,y_train,x_test,y_test, delete = True):
    logreg = LogisticRegression()
    logreg.fit(x_train, y_train)
    Y_pred = logreg.predict(x_test)
    acc_log = round(logreg.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_pred)
    recall = recall_score(y_test,Y_pred)
    if(delete):
        del logreg
        return Y_pred, acc_log, precision, recall
    return logreg,Y_pred,acc_log, precision, recall

# _, acc_log = log_reg(x_train,ang_train, x_test, ang_test)
# print(acc_log)

def svc(x_train,y_train,x_test,y_test, delete = True):
    svc = SVC()
    svc.fit(x_train,y_train)
    Y_svcpred = svc.predict(x_test)
    acc_svc = round(svc.score(x_test, y_test) * 100,2)
    precision = precision_score(y_test, Y_svcpred)
    recall = recall_score(y_test,Y_svcpred)
    if(delete):
        del svc 
        return Y_svcpred, acc_svc,precision, recall
    return svc, Y_svcpred, acc_svc,precision,recall

# _, acc_svc = svc(x_train,ang_train,x_test,ang_test)
# print(acc_svc)

def knn(x_train,y_train,x_test,y_test,n_neighbors = 3,delete = True):
    knn = KNeighborsClassifier(n_neighbors = n_neighbors)
    knn.fit(x_train, y_train)
    Y_knnpred = knn.predict(x_test)
    acc_knn = round(knn.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_knnpred)
    recall = recall_score(y_test,Y_knnpred)
    if(delete):
        del knn
        return Y_knnpred, acc_knn,precision,recall
    return knn, Y_knnpred, acc_knn, precision,recall
# _,acc_knn = knn(x_train,ang_train,x_test,ang_test)
# print(acc_knn)

def gaussian(x_train,y_train,x_test,y_test,delete = True):
    gaussian = GaussianNB()
    gaussian.fit(x_train, y_train)
    Y_gaussianpred = gaussian.predict(x_test)
    acc_gaussian = round(gaussian.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_gaussianpred)
    recall = recall_score(y_test,Y_gaussianpred)
    if(delete):
        del gaussian
        return Y_gaussianpred, acc_gaussian, precision,recall
    return gaussian, Y_gaussianpred, acc_gaussian,precision,recall

# _, acc_gaussian = gaussian(x_train,ang_train,x_test,ang_test)
# print(acc_gaussian)

def perceptron(x_train,y_train,x_test,y_test,delete = True):
    perceptron = Perceptron()
    perceptron.fit(x_train, y_train)
    Y_perceptronpred = perceptron.predict(x_test)
    acc_perceptron = round(perceptron.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_perceptronpred)
    recall = recall_score(y_test,Y_perceptronpred)
    if(delete):
        del perceptron
        return Y_perceptronpred, acc_perceptron,precision,recall
    return perceptron, Y_perceptronpred ,acc_perceptron,precision,recall
# _, acc_perceptron = perceptron(x_train,ang_train,x_test,ang_test)
# print(acc_perceptron)

def linear_svc(x_train,y_train,x_test,y_test,delete = True):
    linear_svc = LinearSVC()
    linear_svc.fit(x_train, y_train)
    Y_linearsvcpred = linear_svc.predict(x_test)
    acc_linear_svc = round(linear_svc.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_linearsvcpred)
    recall = recall_score(y_test,Y_linearsvcpred)
    if(delete):
        del linear_svc
        return Y_linearsvcpred, acc_linear_svc,precision,recall
    return linear_svc, Y_linearsvcpred, acc_linear_svc,precision,recall
# _, acc_linear_svc = linear_svc(x_train,ang_train,x_test,ang_test)
# print(acc_linear_svc)

def sgd(x_train,y_train,x_test,y_test,delete = True):
    sgd = SGDClassifier()
    sgd.fit(x_train, y_train)
    Y_linearsgdpred = sgd.predict(x_test)
    acc_sgd = round(sgd.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_linearsgdpred)
    recall = recall_score(y_test,Y_linearsgdpred)
    if(delete):
        del sgd
        return Y_linearsgdpred , acc_sgd,precision,recall
    return sgd, Y_linearsgdpred, acc_sgd,precision,recall
# _, acc_sgd = sgd(x_train,ang_train,x_test,ang_test)
# print(acc_sgd)

def decision_tree(x_train,y_train,x_test,y_test,delete = True):
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(x_train, y_train)
    Y_dectreepred = decision_tree.predict(x_test)
    acc_decision_tree = round(decision_tree.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_dectreepred)
    recall = recall_score(y_test,Y_dectreepred)
    if(delete):
        del decision_tree
        return Y_dectreepred , acc_decision_tree,precision,recall
    return decision_tree, Y_dectreepred , acc_decision_tree,precision,recall
# _, acc_decision_tree = decision_tree(x_train,ang_train,x_test,ang_test)
# print(acc_decision_tree)

def random_forest(x_train,y_train,x_test,y_test,n_estimators = 100, delete = True):
    random_forest = RandomForestClassifier(n_estimators= n_estimators)
    random_forest.fit(x_train, y_train)
    Y_pred = random_forest.predict(x_test)
    acc_random_forest = round(random_forest.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_pred)
    recall = recall_score(y_test,Y_pred)
    if(delete):
        del random_forest
        return Y_pred, acc_random_forest,precision,recall
    return random_forest, Y_pred, acc_random_forest,precision,recall
# _, acc_random_forest = random_forest(x_train,ang_train, x_test , ang_test)
# print(acc_random_forest)

def get_f1_score(precision,recall):
    return 2 * (precision * recall) / (precision + recall)

def compare_models(x_train, y_train, x_test, y_test):
    _,acc_svc,pre_svc,rec_svc = svc(x_train,y_train,x_test,y_test)
    f1_svc = get_f1_score(pre_svc,rec_svc)
    _,acc_knn,pre_knn,rec_knn = knn(x_train,y_train,x_test,y_test)
    f1_knn = get_f1_score(pre_knn,rec_knn)
    _,acc_log,pre_log,rec_log = log_reg(x_train,y_train,x_test,y_test)
    f1_log = get_f1_score(pre_log,rec_log)
    _,acc_random_forest,pre_rf,rec_rf = random_forest(x_train,y_train,x_test,y_test)
    f1_rf = get_f1_score(pre_rf,rec_rf)
    _,acc_gaussian,pre_gau,rec_gau = gaussian(x_train,y_train,x_test,y_test)
    f1_gau = get_f1_score(pre_gau,rec_gau)
    _,acc_perceptron,pre_per,rec_per = perceptron(x_train,y_train,x_test,y_test)
    f1_per = get_f1_score(pre_per,rec_per)
    _,acc_sgd,pre_sgd,rec_sgd = sgd(x_train,y_train,x_test,y_test)
    f1_sgd =get_f1_score(pre_sgd,rec_sgd)
    _,acc_linear_svc,pre_lsvc,rec_lsvc = linear_svc(x_train,y_train,x_test,y_test)
    f1_lsvc = get_f1_score(pre_lsvc,rec_lsvc)
    _,acc_decision_tree,pre_dtree,rec_dtree =  decision_tree(x_train,y_train,x_test,y_test)
    f1_dtree = get_f1_score(pre_dtree,rec_dtree)
    models = pd.DataFrame({
        'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                  'Random Forest', 'Naive Bayes', 'Perceptron', 
                  'Stochastic Gradient Decent', 'Linear SVC', 
                  'Decision Tree'],
        'Accuracy': [acc_svc, acc_knn, acc_log, 
                  acc_random_forest, acc_gaussian, acc_perceptron, 
                  acc_sgd, acc_linear_svc, acc_decision_tree],
        'Precision': [pre_svc, pre_knn,pre_log,
                      pre_rf,pre_gau,pre_per,pre_sgd,
                      pre_lsvc,pre_dtree],
        'Recall': [rec_svc,rec_knn,rec_log,rec_rf,rec_gau,rec_per,rec_sgd,rec_lsvc,rec_dtree],
        'F1': [f1_svc,f1_knn,f1_log,f1_rf,f1_gau,f1_per,f1_sgd,f1_lsvc,f1_dtree]
    })
    return models[['Model','Accuracy','Precision','Recall','F1']] #.sort_values(by='Score', ascending=False)

def map_for_emotion(y_list, emotional_mapping,emotion):
    return list(map(lambda x: 1 if x == emotional_mapping[emotion] else 0,y_list))

def compare_models_x_emotions(x_train, x_test, emotions):
    result = {'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                  'Random Forest', 'Naive Bayes', 'Perceptron', 
                  'Stochastic Gradient Decent', 'Linear SVC', 
                  'Decision Tree']}
    for emotion,data in emotions.iteritems():
        y_train = data[0]
        y_test = data[1]
        _,acc_svc = svc(x_train,y_train,x_test,y_test)
        _,acc_knn = knn(x_train,y_train,x_test,y_test)
        _,acc_log = log_reg(x_train,y_train,x_test,y_test)
        _,acc_random_forest = random_forest(x_train,y_train,x_test,y_test)
        _,acc_gaussian = gaussian(x_train,y_train,x_test,y_test)
        _,acc_perceptron = perceptron(x_train,y_train,x_test,y_test)
        _,acc_sgd = sgd(x_train,y_train,x_test,y_test)
        _,acc_linear_svc = linear_svc(x_train,y_train,x_test,y_test)
        _,acc_decision_tree =  decision_tree(x_train,y_train,x_test,y_test)
        result[emotion] = [acc_svc, acc_knn, acc_log, 
                  acc_random_forest, acc_gaussian, acc_perceptron, 
                  acc_sgd, acc_linear_svc, acc_decision_tree]
    models = pd.DataFrame(result)
    models = models[['Model'] + list(emotions.iterkeys())]
    return models
        
    



In [155]:
# full = generate_IEMOCAP_df()
data = pd.read_csv('data/IEMOCAP_sentences.csv',index_col=0)
data = data[(data.emotion != 'oth')&(data.emotion != 'xxx')]
emotional_mapping = {'ang': 0, 'sad': 1, 'exc': 2, 'neu': 3,'fru': 4,'hap': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data['emotion_code'] = data['emotion'].map( emotional_mapping ).astype(int)

In [260]:
def get_sample(data,emotional_mapping,emotion):
    emotion_code = emotional_mapping[emotion]
    emotion_sample = data[data.emotion_code == emotion_code]
    sample_size = len(emotion_sample)
    oth_emotion_sample = data[data.emotion_code != emotion_code].sample(n=sample_size)
    sample = pd.concat([oth_emotion_sample,emotion_sample]).sample(frac=1)
    y = map_for_emotion(sample.emotion_code,emotional_mapping,'ang')
    return sample,y

def get_mfcc_feature(filename,n_mfcc):
    y, sr = librosa.load(filename)
    #IPython.display.Audio(data=y,rate=sr)
    mfcc = librosa.feature.mfcc(y=y,sr=sr, n_mfcc=n_mfcc)
    print("Finished "+ filename)
    return np.mean(mfcc.T,axis=0)

def get_rmse(filename):
    y, sr = librosa.load(filename)
    rmse = librosa.feature.rmse(y, frame_length=200)
    print("Finished "+ filename)
    return rmse[0]

def get_zcr(filename):
    y,sr = librosa.load(filename)
    zcr = librosa.feature.zero_crossing_rate(y  + 0.0100) # 0.0200 background noise reduction
    print("Finished "+ filename)
    return zcr[0] 

def get_spectrogram(filename,n_fft = 2048):
    y,sr = librosa.load(filename)
    Y = librosa.stft(y, n_fft=n_fft)
    Ymag = librosa.amplitude_to_db(np.absolute(Y))
    print('Finished '+ filename)
    return np.mean(Ymag.T,axis=0)

def get_melspectrogram(filename,n_fft = 4096):
    y,sr = librosa.load(filename)
    Y = librosa.feature.melspectrogram(y, n_fft=n_fft)
    Ymag = librosa.power_to_db(Y)
    print('Finished '+ filename)
    return np.mean(Ymag,axis=1)

def get_cqt(filename,n_fft = 4096):
    y,sr = librosa.load(filename)
    C = librosa.cqt(y,sr=sr)
    Clog = librosa.amplitude_to_db(np.abs(C))
    print('Finished '+ filename)
    return np.mean(Clog,axis=1)

def get_chroma(filename):
    y,sr = librosa.load(filename)
    chromagram = librosa.feature.chroma_cqt(y,sr=sr)
    print('Finished ' + filename)
    return np.mean(chromagram,axis=1)

def get_spectral_centroid(filename):
    y, sr = librosa.load(filename)
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
    print("Finished "+ filename)
    return spectral_centroids

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def play_audio(filename):
    y, sr = librosa.load(filename)
    return ipd.Audio(data=y,rate=sr)

def get_confusion(y_pred, y_true):
    tp = []
    fp =[]
    fn =[]
    tn = []
    for i in range(len(y_pred)):
        if(y_pred[i] == y_true[i]):
            if y_pred[i] == 1:
                tp.append(True)
                tn.append(False)
            else:
                tn.append(True)
                tp.append(False)
            fp.append(False)
            fn.append(False)
        else:
            if(y_pred[i] == 1):
                fp.append(True)
                fn.append(False)
            else:
                fp.append(False)
                fn.append(True)
            tn.append(False)
            tp.append(False)
            
    return tp,tn,fp,fn

def pad_data(x_train, x_test):
    x_train = pd.DataFrame(x_train).fillna(0)
    x_test = pd.DataFrame(x_test).fillna(0)
    print(x_train.shape, x_test.shape)
    if(x_train.shape[1] > x_test.shape[1]):
        diff = x_train.shape[1] - x_test.shape[1]
        for i in range(diff):
            x_test[x_test.shape[1] + i] = 0
    else:
        diff = x_test.shape[1] - x_train.shape[1]
        for i in range(diff):
            x_train[x_train.shape[1] + i] = 0
    print(x_train.shape, x_test.shape)
    return x_train, x_test

def combine_features(data1,data2):
    return pd.concat([data1, data2],axis=1)


In [300]:
def load_dataset(data, emotion, compute = False ):
    if(compute):
        x,y = get_sample(data,emotional_mapping,'ang')
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        try:
            with open('pickles/'+emotion+'/base_datasets.pickle','wb') as f:
                save = {
                    'X_train' : X_train,
                    'X_test' : X_test,
                    'y_train': y_train,
                    'y_test' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to anger base datasets pickle: ', e)
        return X_train,X_test,y_train,y_test
    else:
        try:
            with open('pickles/'+emotion+'/base_datasets.pickle','rb') as f:
                save = pickle.load(f)
                X_train = save['X_train']
                X_test = save['X_test']
                y_train = save['y_train']
                y_test = save['y_test']
                del save
                return X_train,X_test,y_train,y_test
        except Exception as e:
            print('Error loading base datasets pickle: ', e)

def load_mfcc(emotion, X_train, X_test,y_train,y_test, compute = False):
    if(compute):
        x_train_mfcc = list(map(lambda x: get_mfcc_feature(x,20),X_train['wav_path']))
        x_test_mfcc = list(map(lambda x: get_mfcc_feature(x,20),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/mfcc20.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_mfcc,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_mfcc,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to mfcc pickle: ', e)
        return x_train_mfcc,y_train,x_test_mfcc, y_test
    else:
        try:
            with open('pickles/'+emotion+'/mfcc20.pickle','rb') as f:
                save = pickle.load(f)
                x_train_mfcc = save['train_dataset']
                y_train = save['train_labels']
                x_test_mfcc = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_mfcc,y_train,x_test_mfcc, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_rmse(emotion, X_train,X_test,y_train,y_test,compute = False):
    if(compute):
        x_train_rmse = list(map(lambda x: get_rmse(x),X_train['wav_path']))
        x_test_rmse = list(map(lambda x: get_rmse(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/rmse.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_rmse,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_rmse,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to rmse pickle: ', e)
        return x_train_rmse, y_train, x_test_rmse, y_test
    else:
        try:
            with open('pickles/'+emotion+'/rmse.pickle','rb') as f:
                save = pickle.load(f)
                x_train_rmse = save['train_dataset']
                y_train = save['train_labels']
                x_test_rmse = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_rmse, y_train, x_test_rmse, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_zcr(emotion,X_train,X_test,y_train,y_test,compute= False):
    if(compute):
        x_train_zcr = list(map(lambda x: get_zcr(x),X_train['wav_path']))
        x_test_zcr = list(map(lambda x: get_zcr(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/zcr.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_zcr,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_zcr,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to zcr pickle: ', e)
        return x_train_zcr, y_train, x_test_zcr, y_test
    else:
        try:
            with open('pickles/'+emotion+'/zcr.pickle','rb') as f:
                save = pickle.load(f)
                x_train_zcr = save['train_dataset']
                y_train = save['train_labels']
                x_test_zcr = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_zcr, y_train, x_test_zcr, y_test
        except Exception as e:
            print('Error at:', e)
    
def load_spectro(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train_spectro = list(map(lambda x: get_spectrogram(x),X_train['wav_path']))
        x_test_spectro = list(map(lambda x: get_spectrogram(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/spectrogram.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_spectro,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_spectro,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to spectrogram pickle: ', e)
        return x_train_spectro, y_train, x_test_spectro, y_test
    else:
        try:
            with open('pickles/'+emotion+'/spectrogram.pickle','rb') as f:
                save = pickle.load(f)
                x_train_spectro = save['train_dataset']
                y_train = save['train_labels']
                x_test_spectro = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_spectro, y_train, x_test_spectro, y_test
        except Exception as e:
            print('Error at:', e)
        
def load_melspec(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train_melspec = list(map(lambda x: get_melspectrogram(x),X_train['wav_path']))
        x_test_melspec = list(map(lambda x: get_melspectrogram(x),X_test['wav_path']))
        try:
            with open('pickles/anger/melspectrogram.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_melspec,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_melspec,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to melspectrogram pickle: ', e)
        return x_train_melspec, y_train, x_test_melspec, y_test
    else:
        try:
            with open('pickles/anger/melspectrogram.pickle','rb') as f:
                save = pickle.load(f)
                x_train_melspec = save['train_dataset']
                y_train = save['train_labels']
                x_test_melspec = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_melspec, y_train, x_test_melspec, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_cqt(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train = list(map(lambda x: get_cqt(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_cqt(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/cqt.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to cqt pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/cqt.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_chroma(emotion,X_train, X_test,y_train,y_test, compute=False):
    if(compute):
        x_train = list(map(lambda x: get_chroma(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_chroma(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/chroma_cqt.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to chroma cqt pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/chroma_cqt.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_spectral_centroid(emotion,X_train,X_test,y_train,y_test, compute=False):
    if(compute):
        x_train = list(map(lambda x: get_spectral_centroid(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_spectral_centroid(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/spectral_centroid.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to spectral centroid pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/spectral_centroid.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
    

# Anger Binary Models

In [216]:
X_train,X_test,y_train,y_test = load_dataset(data,'anger')

In [264]:
x_train_mfcc,y_train,x_test_mfcc, y_test = load_mfcc('anger',X_train,X_test,y_train,y_test)
x_train_mfcc = pd.DataFrame(x_train_mfcc,columns=["mfcc_" + str(i+1) for i in range(20)])
x_test_mfcc = pd.DataFrame(x_test_mfcc,columns=["mfcc_"+str(i+1) for i in range(20)])

mfcc_models = compare_models(x_train_mfcc, y_train,x_test_mfcc, y_test)
print('MFCC Anger Model')
mfcc_models

MFCC Anger Model


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,47.96,0.477273,1.000000,0.646154
1,KNN,69.00,0.672986,0.676190,0.674584
2,Logistic Regression,75.79,0.748792,0.738095,0.743405
3,Random Forest,74.66,0.755208,0.690476,0.721393
4,Naive Bayes,72.85,0.727273,0.685714,0.705882
5,Perceptron,56.56,0.950000,0.090476,0.165217
6,Stochastic Gradient Decent,52.04,0.497608,0.990476,0.662420
7,Linear SVC,65.38,0.851852,0.328571,0.474227
8,Decision Tree,69.68,0.675926,0.695238,0.685446


In [253]:
x_train_rmse, y_train, x_test_rmse, y_test = load_rmse('anger',X_train,X_test)
    
x_train_rmse, x_test_rmse = pad_data(x_train_rmse, x_test_rmse)
n_features = x_train_rmse.shape[1]
# column_names = ["rmse_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

rmse_models = compare_models(x_train_rmse, y_train,x_test_rmse, y_test)
rmse_models

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,47.51,0.475113,1.000000,0.644172
1,KNN,69.00,0.695187,0.619048,0.654912
2,Logistic Regression,69.91,0.758389,0.538095,0.629526
3,Random Forest,74.66,0.745000,0.709524,0.726829
4,Naive Bayes,60.41,0.888889,0.190476,0.313725
5,Perceptron,66.97,0.722222,0.495238,0.587571
6,Stochastic Gradient Decent,60.18,0.803571,0.214286,0.338346
7,Linear SVC,65.61,0.716418,0.457143,0.558140
8,Decision Tree,65.16,0.630841,0.642857,0.636792


In [254]:
x_train_zcr, y_train, x_test_zcr, y_test = load_zcr('anger',X_train,X_test)

x_train_zcr, x_test_zcr = pad_data(x_train_zcr, x_test_zcr)
n_features = x_train_zcr.shape[1]
column_names = ["zcr_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

zcr_models = compare_models(x_train_zcr, y_train,x_test_zcr, y_test)
zcr_models

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,47.51,0.475113,1.000000,0.644172
1,KNN,61.54,0.609890,0.528571,0.566327
2,Logistic Regression,66.52,0.658163,0.614286,0.635468
3,Random Forest,70.59,0.688679,0.695238,0.691943
4,Naive Bayes,56.56,0.846154,0.104762,0.186441
5,Perceptron,56.79,0.573643,0.352381,0.436578
6,Stochastic Gradient Decent,49.10,0.482759,1.000000,0.651163
7,Linear SVC,64.71,0.645161,0.571429,0.606061
8,Decision Tree,56.79,0.545894,0.538095,0.541966


In [255]:
x_train_spectro, y_train, x_test_spectro, y_test = load_spectro('anger',X_train,X_test)
spectro_models = compare_models(x_train_spectro, y_train,x_test_spectro, y_test)
spectro_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,65.16,0.595238,0.833333,0.694444
1,KNN,68.78,0.681818,0.642857,0.661765
2,Logistic Regression,71.04,0.695238,0.695238,0.695238
3,Random Forest,72.17,0.723077,0.671429,0.696296
4,Naive Bayes,73.76,0.744792,0.680952,0.711443
5,Perceptron,56.11,0.944444,0.080952,0.149123
6,Stochastic Gradient Decent,58.37,0.861111,0.147619,0.252033
7,Linear SVC,62.44,0.560109,0.976190,0.711806
8,Decision Tree,62.22,0.601896,0.604762,0.603325


In [279]:
x_train_melspec, y_train, x_test_melspec, y_test = load_melspec('anger',X_train,X_test)
melspec_models = compare_models(x_train_melspec, y_train,x_test_melspec, y_test)
melspec_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,61.99,0.559322,0.942857,0.702128
1,KNN,68.78,0.671429,0.671429,0.671429
2,Logistic Regression,77.15,0.768473,0.742857,0.755448
3,Random Forest,71.49,0.712121,0.671429,0.691176
4,Naive Bayes,72.40,0.747191,0.633333,0.685567
5,Perceptron,59.28,0.788462,0.195238,0.312977
6,Stochastic Gradient Decent,47.51,0.475113,1.000000,0.644172
7,Linear SVC,54.75,0.512563,0.971429,0.671053
8,Decision Tree,67.42,0.668367,0.623810,0.645320


In [257]:
x_train_cqt, y_train, x_test_cqt, y_test = load_cqt('anger',X_train,X_test,y_train,y_test)
cqt_models = compare_models(x_train_cqt, y_train,x_test_cqt, y_test)
cqt_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,60.18,0.549133,0.904762,0.683453
1,KNN,65.84,0.629956,0.680952,0.654462
2,Logistic Regression,70.81,0.723757,0.623810,0.670077
3,Random Forest,71.04,0.709184,0.661905,0.684729
4,Naive Bayes,69.00,0.708571,0.590476,0.644156
5,Perceptron,55.20,1.000000,0.057143,0.108108
6,Stochastic Gradient Decent,66.74,0.760331,0.438095,0.555891
7,Linear SVC,52.94,1.000000,0.009524,0.018868
8,Decision Tree,65.61,0.636792,0.642857,0.639810


In [258]:
x_train_chroma, y_train, x_test_chroma, y_test = load_chroma('anger',X_train,X_test,y_train,y_test)
chroma_models = compare_models(x_train_chroma, y_train,x_test_chroma, y_test)
chroma_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,60.41,0.572016,0.661905,0.613687
1,KNN,56.11,0.533898,0.600000,0.565022
2,Logistic Regression,60.41,0.579909,0.604762,0.592075
3,Random Forest,56.79,0.547739,0.519048,0.533007
4,Naive Bayes,61.76,0.580392,0.704762,0.636559
5,Perceptron,54.07,0.888889,0.038095,0.073059
6,Stochastic Gradient Decent,61.99,0.610526,0.552381,0.580000
7,Linear SVC,62.67,0.600000,0.642857,0.620690
8,Decision Tree,57.01,0.546729,0.557143,0.551887


In [265]:
x_train_specentr, y_train, x_test_specentr, y_test = load_spectral_centroid('anger',X_train,X_test,y_train,y_test)
x_train_specentr, x_test_specentr = pad_data(x_train_specentr, x_test_specentr)
n_features = x_train_specentr.shape[1]
# column_names = ["spectralCent_" + str(i+1) for i in range(n_features)]
# x_train_specentr = pd.DataFrame(x_train_specentr,columns=column_names)
# x_test_specentr = pd.DataFrame(x_test_specentr,columns=column_names)

specentr_models = compare_models(x_train_specentr, y_train,x_test_specentr, y_test)
specentr_models

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,47.51,0.475113,1.000000,0.644172
1,KNN,57.01,0.558824,0.452381,0.500000
2,Logistic Regression,53.17,0.506329,0.571429,0.536913
3,Random Forest,65.38,0.633803,0.642857,0.638298
4,Naive Bayes,47.51,0.474771,0.985714,0.640867
5,Perceptron,53.17,1.000000,0.014286,0.028169
6,Stochastic Gradient Decent,47.06,0.471564,0.947619,0.629747
7,Linear SVC,53.62,0.666667,0.047619,0.088889
8,Decision Tree,56.56,0.540541,0.571429,0.555556


In [276]:
x_train_combine1 = combine_features(x_train_mfcc,x_train_rmse)
x_test_combine1 = combine_features(x_test_mfcc,x_test_rmse)
combine1_models = compare_models(x_train_combine1,y_train,x_test_combine1,y_test)
print('MFCC + RMSE')
combine1_models


/usr/local/Cellar/ipython@5/5.7.0/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars


MFCC + RMSE


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,50.45,0.527363,0.460870,0.491879
1,KNN,50.23,0.524038,0.473913,0.497717
2,Logistic Regression,50.45,0.530726,0.413043,0.464548
3,Random Forest,53.85,0.560185,0.526087,0.542601
4,Naive Bayes,52.49,0.524631,0.926087,0.669811
5,Perceptron,52.04,0.520362,1.000000,0.684524
6,Stochastic Gradient Decent,52.04,0.520362,1.000000,0.684524
7,Linear SVC,47.96,0.000000,0.000000,NaN
8,Decision Tree,52.26,0.542222,0.530435,0.536264


In [274]:
x_train_combine2 = combine_features(x_train_combine1,x_train_zcr)
x_test_combine2 = combine_features(x_test_combine1,x_test_zcr)
combine2_models = compare_models(x_train_combine2,y_train,x_test_combine2,y_test)
print('MFCC + RMSE + ZCR')
combine2_models

MFCC + RMSE + ZCR


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,52.71,0.547511,0.526087,0.536585
1,KNN,50.45,0.526316,0.478261,0.501139
2,Logistic Regression,48.42,0.505155,0.426087,0.462264
3,Random Forest,51.58,0.538462,0.486957,0.511416
4,Naive Bayes,52.49,0.524631,0.926087,0.669811
5,Perceptron,52.04,0.520362,1.000000,0.684524
6,Stochastic Gradient Decent,52.04,0.520362,1.000000,0.684524
7,Linear SVC,47.96,0.000000,0.000000,NaN
8,Decision Tree,50.90,0.526531,0.560870,0.543158


In [297]:
# x_train_melspec = pd.DataFrame(x_train_melspec)
# x_test_melspec = pd.DataFrame(x_test_melspec)
x_train_combine3 = combine_features(x_train_combine2,x_train_melspec)
x_test_combine3 = combine_features(x_test_combine2,x_test_melspec)
combine3_models = compare_models(x_train_combine3,y_train,x_test_combine3,y_test)
print('MFCC + RMSE + ZCR + MELSPEC')
combine3_models

MFCC + RMSE + ZCR + MELSPEC


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,74.21,0.733010,0.719048,0.725962
1,KNN,67.87,0.663462,0.657143,0.660287
2,Logistic Regression,77.38,0.772277,0.742857,0.757282
3,Random Forest,75.79,0.746411,0.742857,0.744630
4,Naive Bayes,60.86,0.849057,0.214286,0.342205
5,Perceptron,60.41,0.872340,0.195238,0.319066
6,Stochastic Gradient Decent,68.33,0.836538,0.414286,0.554140
7,Linear SVC,61.99,0.556757,0.980952,0.710345
8,Decision Tree,62.67,0.605634,0.614286,0.609929


In [271]:
# X_test[['text','emotion']].iloc[fp].head(10)
# play_audio(X_test.iloc[fp]['wav_path'].values[3])

# Sadness Binary Models

In [298]:
X_train,X_test,y_train,y_test = load_dataset(data,'sadness',True)

In [301]:
x_train_mfcc,y_train,x_test_mfcc, y_test = load_mfcc('sadness',X_train,X_test,y_train,y_test,True)
x_train_mfcc = pd.DataFrame(x_train_mfcc,columns=["mfcc_" + str(i+1) for i in range(20)])
x_test_mfcc = pd.DataFrame(x_test_mfcc,columns=["mfcc_"+str(i+1) for i in range(20)])

mfcc_models = compare_models(x_train_mfcc, y_train,x_test_mfcc, y_test)
print('MFCC Sadness Model')
mfcc_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M021.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro06/Ses01F_impro06_F003.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F009.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_1/Ses05F_script02_1_M

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro08/Ses05F_impro08_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro06/Ses05F_impro06_F027.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/

...

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_F013.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M036.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05b/Ses03M_impro05b_M022.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_2/Ses05F_script01_2_M014.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_1/Ses01M_script02_1_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_M001.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/S

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F001.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses04M_script01_2_F015.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F036.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impr

/usr/local/Cellar/ipython@5/5.7.0/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars


MFCC Sadness Model


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,48.19,0.000000,0.000000,NaN
1,KNN,71.72,0.742991,0.694323,0.717833
2,Logistic Regression,73.30,0.763033,0.703057,0.731818
3,Random Forest,74.43,0.763636,0.733624,0.748330
4,Naive Bayes,70.59,0.726027,0.694323,0.709821
5,Perceptron,69.00,0.794872,0.541485,0.644156
6,Stochastic Gradient Decent,51.81,0.518100,1.000000,0.682563
7,Linear SVC,53.39,0.960000,0.104803,0.188976
8,Decision Tree,64.71,0.652720,0.681223,0.666667


In [303]:
x_train_rmse, y_train, x_test_rmse, y_test = load_rmse('sadness',X_train,X_test,y_train,y_test,True)
    
x_train_rmse, x_test_rmse = pad_data(x_train_rmse, x_test_rmse)
n_features = x_train_rmse.shape[1]
# column_names = ["rmse_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

rmse_models = compare_models(x_train_rmse, y_train,x_test_rmse, y_test)
rmse_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M021.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_F001.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro06/Ses01F_impro06_F003.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro08/Ses05F_impro08_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro06/Ses05F_impro06_F027.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/

...

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro08/Ses05M_impro08_F027.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro01/Ses02M_impro01_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_1/Ses04F_script01_1_M020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script02_2/Ses02F_script02_2_M037.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro04/Ses01M_impro04_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F018.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F007.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F042.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F040.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro04/Ses01F_impro04_M035.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_2/Ses05F_script02_2_F033.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro07/Ses04M_impro07_M011.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M016.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M012.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M004.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro02/Ses01M_impro02_M011.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro07/Ses01M_impro07_F020.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro05/Ses04M_impro05_M010.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M023.wav
Finish

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_1/Ses01M_script02_1_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_M001.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_M013.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_M029.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1/Ses05M_script01_1_F021.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F024.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F015.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F003.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro05/Ses02F_impro05_M016.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F025.wa

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,50.00,1.000000,0.034934,0.067511
1,KNN,64.48,0.768657,0.449782,0.567493
2,Logistic Regression,65.61,0.758389,0.493450,0.597884
3,Random Forest,70.81,0.727273,0.698690,0.712695
4,Naive Bayes,54.30,0.846154,0.144105,0.246269
5,Perceptron,65.16,0.741935,0.502183,0.598958
6,Stochastic Gradient Decent,65.16,0.790698,0.445415,0.569832
7,Linear SVC,64.48,0.757143,0.462882,0.574526
8,Decision Tree,59.73,0.624390,0.558952,0.589862


In [304]:
x_train_zcr, y_train, x_test_zcr, y_test = load_zcr('sadness',X_train,X_test,y_train,y_test,True)

x_train_zcr, x_test_zcr = pad_data(x_train_zcr, x_test_zcr)
n_features = x_train_zcr.shape[1]
column_names = ["zcr_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

zcr_models = compare_models(x_train_zcr, y_train,x_test_zcr, y_test)
zcr_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M021.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro06/Ses01F_impro06_F003.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F009.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_1/Ses05F_script02_1_M

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro08/Ses05F_impro08_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro06/Ses05F_impro06_F027.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/

...

Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro01/Ses02M_impro01_M007.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_M027.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_F012.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M018.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro04/Ses01F_impro04_M035.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_2/Ses05F_script02_2_F033.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro07/Ses04M_impro07_M011.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M016.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M012.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M004.wav
Finish

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_F013.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M036.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05b/Ses03M_impro05b_M022.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_2/Ses05F_script01_2_M014.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_1/Ses01M_script02_1_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_M001.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/S

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05b/Ses03M_impro05b_F005.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F001.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses04M_script01_2_F015.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F036.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script

/usr/local/Cellar/ipython@5/5.7.0/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,48.19,0.000000,0.000000,NaN
1,KNN,62.90,0.699387,0.497817,0.581633
2,Logistic Regression,67.65,0.712871,0.628821,0.668213
3,Random Forest,72.40,0.731602,0.737991,0.734783
4,Naive Bayes,51.58,0.826087,0.082969,0.150794
5,Perceptron,64.25,0.622837,0.786026,0.694981
6,Stochastic Gradient Decent,62.44,0.733333,0.432314,0.543956
7,Linear SVC,66.06,0.700508,0.602620,0.647887
8,Decision Tree,63.35,0.651584,0.628821,0.640000


In [305]:
x_train_spectro, y_train, x_test_spectro, y_test = load_spectro('sadness',X_train,X_test,y_train,y_test,True)
spectro_models = compare_models(x_train_spectro, y_train,x_test_spectro, y_test)
spectro_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M034.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M024.

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F009.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_1/Ses05F_script02_1_M024.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script

Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro02/Ses04F_impro02_F018.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro03/Ses02M_impro03_F023.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_M043.wav
Finished d

...

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro08/Ses05M_impro08_F027.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro01/Ses02M_impro01_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_1/Ses04F_script01_1_M020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script02_2/Ses02F_script02_2_M037.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro04/Ses01M_impro04_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F018.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F007.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F042.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F040.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro04/Ses01F_impro04_M035.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_2/Ses05F_script02_2_F033.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro07/Ses04M_impro07_M011.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M016.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M012.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M004.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro02/Ses01M_impro02_M011.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro07/Ses01M_impro07_F020.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro05/Ses04M_impro05_M010.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M023.wav
Finish

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_1/Ses01M_script02_1_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_M001.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_M013.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_M029.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1/Ses05M_script01_1_F021.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F024.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F036.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F015.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F003.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro05/Ses02F_impro05_M016.wa

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,67.87,0.682008,0.711790,0.696581
1,KNN,68.55,0.725000,0.633188,0.675991
2,Logistic Regression,69.23,0.722488,0.659389,0.689498
3,Random Forest,72.85,0.768473,0.681223,0.722222
4,Naive Bayes,69.46,0.737374,0.637555,0.683841
5,Perceptron,51.81,0.518100,1.000000,0.682563
6,Stochastic Gradient Decent,51.81,0.518100,1.000000,0.682563
7,Linear SVC,69.00,0.645570,0.890830,0.748624
8,Decision Tree,69.23,0.712329,0.681223,0.696429


In [306]:
x_train_melspec, y_train, x_test_melspec, y_test = load_melspec('sadness',X_train,X_test,y_train,y_test,True)
melspec_models = compare_models(x_train_melspec, y_train,x_test_melspec, y_test)
melspec_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M021.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_F001.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro06/Ses01F_impro06_F003.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro08/Ses05F_impro08_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro06/Ses05F_impro06_F027.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/

...

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_1/Ses04F_script01_1_M020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script02_2/Ses02F_script02_2_M037.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro04/Ses01M_impro04_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F018.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F007.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F042.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F040.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_F032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro03/Ses03M_impro03_M017.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script02_2/Ses02F_sc

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro07/Ses04M_impro07_M011.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M016.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M012.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M004.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro02/Ses01M_impro02_M011.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro07/Ses01M_impro07_F020.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro05/Ses04M_impro05_M010.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M023.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F002.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F006.wav


Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/Ses01F_script02_1_M013.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_M029.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1/Ses05M_script01_1_F021.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F024.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses04M_script01_2_F013.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script01_3/Ses02M_script01_3_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F015.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F003.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro05/Ses02F_impro05_M016.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F025.wa

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,67.19,0.636364,0.855895,0.729981
1,KNN,69.00,0.712963,0.672489,0.692135
2,Logistic Regression,71.49,0.751220,0.672489,0.709677
3,Random Forest,71.72,0.765306,0.655022,0.705882
4,Naive Bayes,68.78,0.735751,0.620087,0.672986
5,Perceptron,69.23,0.839416,0.502183,0.628415
6,Stochastic Gradient Decent,52.04,0.519451,0.991266,0.681682
7,Linear SVC,69.00,0.783951,0.554585,0.649616
8,Decision Tree,67.42,0.699531,0.650655,0.674208
